In [1]:
import toml

### Load demo Pipfile

In [7]:
pipfile = toml.load("../Pipfile")

In [9]:
pipfile.keys()

dict_keys(['source', 'requires', 'packages', 'dev-packages'])

In [10]:
pipfile

{'source': [{'url': 'https://pypi.python.org/simple',
   'verify_ssl': True,
   'name': 'pypi'}],
 'requires': {'python_version': '2.7'},
 'packages': {'requests': {'extras': ['socks']},
  'records': '>0.5.0',
  'django': {'git': 'https://github.com/django/django.git',
   'ref': '1.11.4',
   'editable': True},
  'e682b37': {'file': 'https://github.com/divio/django-cms/archive/release/3.4.x.zip'},
  'e1839a8': {'path': '.', 'editable': True},
  'pywinusb': {'version': '*', 'os_name': "=='nt'", 'index': 'pypi'}},
 'dev-packages': {'nose': '*',
  'unittest2': {'version': '>=1.0,<3.0',
   'markers': "python_version < '2.7.9' or (python_version >= '3.0' and python_version < '3.4')"}}}

### Load Pyproject

In [3]:
pyproject = toml.load("../pyproject.toml")

In [5]:
poetry = pyproject['tool']['poetry']

In [6]:
poetry

{'name': 'poetrip',
 'version': '0.1.0',
 'description': 'Generate Pipfile from pyproject.toml',
 'license': 'MIT',
 'authors': ['Joffrey Bienvenu <joffreybvn@gmail.com>'],
 'readme': 'README.md',
 'repository': 'https://github.com/joffreybvn/poetrip',
 'homepage': 'https://github.com/joffreybvn/poetrip',
 'keywords': ['packaging', 'poetry', 'pipenv', 'pipfile'],
 'scripts': {'poetrip': 'poetrip.main:app'},
 'dependencies': {'python': '^3.6', 'typer': '^0.6.1', 'toml': '^0.9'},
 'dev-dependencies': {'jupyter': '*', 'pytest': '^3.0', 'pytest-cov': '^2.4'}}

In [ ]:
dependencies -> packages
dev-dependencies -> dev-packages


In [21]:
import toml
from typing import Dict, Any


class PyProject:

    def __init__(
            self,
            infos: dict = None,
            dependencies: dict = None,
            dev_dependencies: dict = None
    ):
        self._infos = infos or {}
        self._dependencies = dependencies or {}
        self._dev_dependencies = dev_dependencies or {}

    @classmethod
    def from_file(cls, filename: str = 'pyproject.toml') -> 'PyProject':
        """Load a PyProject file from a filename."""
        toml_content = dict(toml.load(filename))
        poetry_attributes: dict = toml_content['tool']['poetry']

        return cls(
            dependencies=poetry_attributes.pop('dependencies'),
            dev_dependencies=poetry_attributes.pop('dev-dependencies'),
            infos=poetry_attributes
        )

    @property
    def attributes(self) -> Dict[str, Any]:
        return {'tool': {'poetry': {
            **self._infos,
            'dependencies': self._dependencies,
            'dev-dependencies': self._dev_dependencies
        }}}

    def _get_requires(self) -> dict:
        python_version: str = self._dependencies.pop('python')
        return {'python-version': python_version}

    def to_pipfile(self) -> 'PipFile':
        return PipFile(
            source=self._infos,
            requires=self._get_requires(),
            packages=self._dependencies,
            dev_packages=self._dev_dependencies
        )


class PipFile:

    def __init__(
            self,
            source: dict = None,
            requires: dict = None,
            packages: dict = None,
            dev_packages: dict = None
    ):
        self._source: dict = source or {}
        self._requires: dict = requires or {}
        self._packages: dict = packages or {}
        self._dev_packages: dict = dev_packages or {}

    @property
    def attributes(self) -> Dict[str, Any]:
        return {
            'source': [self._source],
            'requires': self._requires,
            'packages': self._packages,
            'dev-packages': self._dev_packages
        }

    def to_file(self, filename: str = 'Pipfile') -> None:
        """Create a Pipfile on disk."""
        with open(filename, 'w') as file:
            toml.dump(self.attributes, file)

In [22]:
pyproject = PyProject.from_file('../pyproject.toml')
pipfile = pyproject.to_pipfile()
pipfile.to_file()

{'source': [{'name': 'poetrip',
   'version': '0.1.0',
   'description': 'Generate Pipfile from pyproject.toml',
   'license': 'MIT',
   'authors': ['Joffrey Bienvenu <joffreybvn@gmail.com>'],
   'readme': 'README.md',
   'repository': 'https://github.com/joffreybvn/poetrip',
   'homepage': 'https://github.com/joffreybvn/poetrip',
   'keywords': ['packaging', 'poetry', 'pipenv', 'pipfile'],
   'scripts': {'poetrip': 'poetrip.main:app'}}],
 'requires': {'python-version': '^3.6'},
 'packages': {'typer': '^0.6.1', 'toml': '^0.9'},
 'dev-packages': {'jupyter': '*', 'pytest': '^3.0', 'pytest-cov': '^2.4'}}